## Stuff Document Chain

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("apjspeech.pdf")
docs = loader.load()
docs

[Document(metadata={'source': 'apjspeech.pdf', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowi

In [2]:
from langchain_core.prompts import PromptTemplate

template = """ Write a concise and short summary of the following speech, 
speech :{text}

"""

prompt = PromptTemplate(input_variables=["text"], template=template)

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq

llm = ChatGroq(model='gemma2-9b-it', api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000011FB28D8040>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000011FB28D8D00>, model_name='gemma2-9b-it', groq_api_key=SecretStr('**********'))

In [4]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type="stuff",prompt= prompt, verbose=True)

output_summary = chain.run(docs)
output_summary

c:\Users\hp\OneDrive\Desktop\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 Write a concise and short summary of the following speech, 
speech :A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online i

'In his departing speech, A.P.J. Abdul Kalam reflects on his five years as President of India, highlighting key experiences and messages he gleaned from interactions across the country.  \n\nHe emphasizes the aspirations of India\'s youth for a developed nation by 2020, a goal he fully endorses. \n\nKalam stresses the importance of:\n\n* **Empowering villages:** Sharing his experience of a successful village in Nagaland, he advocates for connected, prosperous rural communities.\n* **Mobilizing rural core competence:** He highlights the Periyar PURA model, showcasing how technology and collaboration can drive economic growth in rural areas.\n* **"Seed to food" approach for agricultural growth:** Inspired by farmers he met, Kalam promotes doubling agricultural production through sustainable practices.\n* **Overcoming challenges through partnership:** He celebrates the resilience of the people of Urusa village in Jammu & Kashmir after the earthquake and the Car Nicobar islanders after the

# Map Reduce to Summarize Large Documents

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
docs

[Document(metadata={'source': 'apjspeech.pdf', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowi

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
texts = text_splitter.split_documents(docs)
texts

[Document(metadata={'source': 'apjspeech.pdf', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowi

In [9]:
len(texts)

13

In [10]:
chunk_prompt = """ 
Please summarize the below speech:
Speech:{text}
Summary:
"""

map_reduce_template = PromptTemplate(
    input_variables=["text"],
    template=chunk_prompt,
)

In [12]:
final_prompt = """ Provide a final summary of the entire speech with these important points.
Add a motivation title, Start with precise summary with an introduction and provide the summary in numbers 
points for the speech.
Speech:{text}
"""

final_prompt_template = PromptTemplate(
    input_variables=["text"], template=final_prompt
)
final_prompt_template

PromptTemplate(input_variables=['text'], template=' Provide a final summary of the entire speech with these important points.\nAdd a motivation title, Start with precise summary with an introduction and provide the summary in numbers \npoints for the speech.\nSpeech:{text}\n')

In [ ]:
summary_chain  = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt = map_reduce_template,
    combine_prompt = final_prompt_template,
)

output = summary_chain.run(texts)
output

## Refine Chain type

In [16]:
chain = load_summarize_chain(
    llm= llm,
    chain_type = "refine",
    verbose= True
)
refine_output= chain.run(texts)

refine_output



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have man

'A.P.J. Abdul Kalam\'s departing speech paints a vivid picture of his vision for a developed India by 2020.  He emphasizes the crucial role of youth aspirations, empowered villages, and a thriving agricultural sector in driving this progress.  Kalam stresses the importance of overcoming challenges through cooperation, courage, and connectivity, drawing inspiration from his interactions with diverse groups, including young people, villagers, and farmers.  \n\nHe highlights the remarkable resilience of Indian citizens, citing examples like the people of Car Nicobar Islands who embraced tsunami victims and the villagers of Khuzama in Nagaland who actively participated in relief efforts.  Kalam champions village development through initiatives like "PURA," which aims to bridge the rural-urban divide by providing urban amenities in rural areas, fostering economic growth and job creation.  He also emphasizes "Seed to Food" initiatives to transform India\'s agriculture by empowering farmers a